# Previsão do Preço de Casas Com Estimators

Base de dados retirada do Kaggle, disponível neste [Link](https://www.kaggle.com/harlfoxem/housesalesprediction).

In [19]:
import pandas as pd
url = 'https://raw.githubusercontent.com/diegogrr/UdemyTF/master/regressao_linear/original.csv'
base = pd.read_csv(url)
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


Visualizar a quantidade de registros:

In [20]:
base.shape

(21613, 21)

In [0]:
x = base.iloc[:,5:6].values # metragem quadrada
y = base.iloc[:,2:3].values # preço das casas

Escalonamento:

In [22]:
from sklearn.preprocessing import StandardScaler
# metragem quadrada
scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)
print(x)
print()
#preço da casa
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)
print(y)

[[-0.97983502]
 [ 0.53363434]
 [-1.42625404]
 ...
 [-1.15404732]
 [-0.52252773]
 [-1.15404732]]

[[-0.86671733]
 [-0.00568792]
 [-0.98084935]
 ...
 [-0.37586519]
 [-0.38158814]
 [-0.58588173]]


Importar TensorFlow:

In [23]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.__version__

'2.2.0'

Criar uma lista com as colunas:

In [24]:
colunas = [tf.feature_column.numeric_column('x',shape=[1])]
print(colunas)

[NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


Criar o regressor:

Ainda não estamos trabalhando com redes neurais.

In [25]:
regressor = tf.estimator.LinearRegressor(feature_columns=colunas)
# normal aparecer alguns warnings

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp6ya3tc6w', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Divisão da Base de Dados

Entre treinamento e teste.


*   70% para o treinamento
*   30% para o teste



In [0]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.3)

Registros para o treinamento (aprendizado):

In [27]:
print(x_treinamento.shape)
print(y_treinamento.shape)

(15129, 1)
(15129, 1)


Registros para os testes (generalização):

In [28]:
print(x_teste.shape, y_teste.shape)

(6484, 1) (6484, 1)


## Função de Treinamento

Conceito de função dos estimators: a origem é 'pandas', mas lembre-se que nós convertemos para matrizes do tipo 'numpy'.

In [0]:
funcao_treinamento = tf.estimator.inputs.numpy_input_fn({'x': x_treinamento}, y_treinamento, batch_size=32, 
                                                        num_epochs=None, shuffle=True)

## Função de Teste

In [0]:
funcao_teste = tf.estimator.inputs.numpy_input_fn({'x': x_teste}, y_teste, batch_size=32, 
                                                  num_epochs=1000, shuffle=False)
# shuffle=True pois não queremos que seja aleatório, 
# e sim que seja feito o teste em todos os registro

## Treinamento do Regressor

In [31]:
regressor.train(input_fn=funcao_treinamento, steps=10000)
# steps - ideia dos passos da descida do gradiente

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Please use `layer.add_weight` method instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp6ya3tc6w/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving c

### Para visualizar algumas métricas

In [32]:
metricas_treinamento = regressor.evaluate(input_fn=funcao_treinamento, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-22T15:42:40Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6ya3tc6w/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 9.95610s
INFO:tensorflow:Finished evaluation at 2020-05-22-15:42:50
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.4892956, global_step = 10000, label/mean = -0.0033089374, loss = 15.657459, prediction/mean = 0.00776

Métricas da base de teste:

In [33]:
metricas_teste = regressor.evaluate(input_fn=funcao_teste, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-22T15:44:02Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6ya3tc6w/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 10.29179s
INFO:tensorflow:Finished evaluation at 2020-05-22-15:44:12
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.55031896, global_step = 10000, label/mean = 0.0075356276, loss = 17.610207, prediction/mean = 0.0110

Realizando algumas comparações:

Em geral, o item 'loss' será melhor no treinamento do que no teste.

In [35]:
print(metricas_treinamento)
print(metricas_teste)

{'average_loss': 0.4892956, 'label/mean': -0.0033089374, 'loss': 15.657459, 'prediction/mean': 0.0077664442, 'global_step': 10000}
{'average_loss': 0.55031896, 'label/mean': 0.0075356276, 'loss': 17.610207, 'prediction/mean': 0.011026491, 'global_step': 10000}


Nosso modelo está pronto!

## Previsões

In [0]:
import numpy as np

Objetivo: prever o preço das casas baseada na metragem informada abaixo:

In [38]:
novas_casas = np.array([[800],[900],[1000]])
print(novas_casas)

[[ 800]
 [ 900]
 [1000]]


Fazer o escalonamento:

In [39]:
novas_casas = scaler_x.transform(novas_casas)
print(novas_casas)

[[-1.39358923]
 [-1.28470655]
 [-1.17582386]]


Criar uma função para fazer a previsão desses novos registros:

In [0]:
funcao_previsao = tf.estimator.inputs.numpy_input_fn({'x': novas_casas}, shuffle=False)

In [0]:
previsoes = regressor.predict(input_fn=funcao_previsao)

In [43]:
list(previsoes) # predictions está escalonado

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6ya3tc6w/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([-0.9384173], dtype=float32)},
 {'predictions': array([-0.86441296], dtype=float32)},
 {'predictions': array([-0.7904085], dtype=float32)}]

Faremos um `for` para percorrer esses generators e obter o valor das casas em dolar, e não escalonados.

In [44]:
for p in regressor.predict(input_fn=funcao_previsao):
  #print(p['predictions'])
  print(scaler_y.inverse_transform(p['predictions']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6ya3tc6w/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[195577.58]
[222745.98]
[249914.39]


Observe que conforme a metragem aumenta, também aumenta o preço da casa.